# Implementing binary decision trees

The goal of this notebook is to implement your own binary decision tree classifier. You will:

- Use SFrames to do some feature engineering.
- Transform categorical variables into binary variables.
- Write a function to compute the number of misclassified examples in an intermediate node.
- Write a function to find the best feature to split on.
- Build a binary decision tree from scratch.
- Make predictions using the decision tree.
- Evaluate the accuracy of the decision tree.
- Visualize the decision at the root node.

In [2]:
import graphlab
import pandas as pd
import numpy as np
import json
from __future__ import division

In [3]:
loans = graphlab.SFrame('lending-club-data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Santosh\AppData\Local\Temp\graphlab_server_1482625756.log.0


This non-commercial license of GraphLab Create for academic use is assigned to santosh.chilkunda@gmail.com and will expire on July 20, 2017.


In [4]:
loans.head(5)

Columns:
	id	int
	member_id	int
	loan_amnt	int
	funded_amnt	int
	funded_amnt_inv	int
	term	str
	int_rate	float
	installment	float
	grade	str
	sub_grade	str
	emp_title	str
	emp_length	str
	home_ownership	str
	annual_inc	int
	is_inc_v	str
	issue_d	str
	loan_status	str
	pymnt_plan	str
	url	str
	desc	str
	purpose	str
	title	str
	zip_code	str
	addr_state	str
	dti	float
	delinq_2yrs	int
	earliest_cr_line	str
	inq_last_6mths	int
	mths_since_last_delinq	int
	mths_since_last_record	int
	open_acc	int
	pub_rec	int
	revol_bal	int
	revol_util	float
	total_acc	int
	initial_list_status	str
	out_prncp	float
	out_prncp_inv	float
	total_pymnt	float
	total_pymnt_inv	float
	total_rec_prncp	float
	total_rec_int	float
	total_rec_late_fee	float
	recoveries	float
	collection_recovery_fee	float
	last_pymnt_d	str
	last_pymnt_amnt	float
	next_pymnt_d	str
	last_credit_pull_d	str
	collections_12_mths_ex_med	int
	mths_since_last_major_derog	str
	policy_code	int
	not_compliant	int
	status	str
	inactive_loans	int
	bad_loans	int
	emp_length_num	int
	grade_num	int
	sub_grade_num	float
	delinq_2yrs_zero	int
	pub_rec_zero	int
	collections_12_mths_zero	int
	short_emp	int
	payment_inc_ratio	float
	final_d	str
	last_delinq_none	int
	last_record_none	int
	last_major_derog_none	int

Rows: 5

Data:
+---------+-----------+-----------+-------------+-----------------+------------+
|    id   | member_id | loan_amnt | funded_amnt | funded_amnt_inv |    term    |
+---------+-----------+-----------+-------------+-----------------+------------+
| 1077501 |  1296599  |    5000   |     5000    |       4975      |  36 months |
| 1077430 |  1314167  |    2500   |     2500    |       2500      |  60 months |
| 1077175 |  1313524  |    2400   |     2400    |       2400      |  36 months |
| 1076863 |  1277178  |   10000   |    10000    |      10000      |  36 months |
| 1075269 |  1311441  |    5000   |     5000    |       5000      |  36 months |
+---------+-----------+-----------+-------------+-----------------+------------+
+----------+-------------+-------+-----------+----------------------+------------+
| int_rate | installment | grade | sub_grade |      emp_title       | emp_length |
+----------+-------------+-------+-----------+----------------------+------------+
|  10.65   |    162.87   |   B   |     B2    |                      | 10+ years  |
|  15.27   |    59.83    |   C   |     C4    |        Ryder         |  < 1 year  |
|  15.96   |    84.33    |   C   |     C5    |                      | 10+ years  |
|  13.49   |    339.31   |   C   |     C1    | AIR RESOURCES BOARD  | 10+ years  |
|   7.9    |    156.46   |   A   |     A4    | Veolia Transportaton |  3 years   |
+----------+-------------+-------+-----------+----------------------+------------+
+----------------+------------+-----------------+-----------------+-------------+
| home_ownership | annual_inc |     is_inc_v    |     issue_d     | loan_status |
+----------------+------------+-----------------+-----------------+-------------+
|      RENT      |   24000    |     Verified    | 20111201T000000 |  Fully Paid |
|      RENT      |   30000    | Source Verified | 20111201T000000 | Charged Off |
|      RENT      |   12252    |   Not Verified  | 20111201T000000 |  Fully Paid |
|      RENT      |   49200    | Source Verified | 20111201T000000 |  Fully Paid |
|      RENT      |   36000    | Source Verified | 20111201T000000 |  Fully Paid |
+----------------+------------+-----------------+-----------------+-------------+
+------------+-------------------------------+-------------------------------+-----+
| pymnt_plan |              url              |              desc             | ... |
+------------+-------------------------------+-------------------------------+-----+
|     n      | https://www.lendingclub.co... |   Borrower added on 12/22/... | ... |
|     n      | https://www.lendingclub.co... |   Borrower added on 12/22/... | ... |
|     n      | https://www.lendingclub.co... |                               | ... 

In [5]:
loans.column_names

<bound method SFrame.column_names of Columns:
	id	int
	member_id	int
	loan_amnt	int
	funded_amnt	int
	funded_amnt_inv	int
	term	str
	int_rate	float
	installment	float
	grade	str
	sub_grade	str
	emp_title	str
	emp_length	str
	home_ownership	str
	annual_inc	int
	is_inc_v	str
	issue_d	str
	loan_status	str
	pymnt_plan	str
	url	str
	desc	str
	purpose	str
	title	str
	zip_code	str
	addr_state	str
	dti	float
	delinq_2yrs	int
	earliest_cr_line	str
	inq_last_6mths	int
	mths_since_last_delinq	int
	mths_since_last_record	int
	open_acc	int
	pub_rec	int
	revol_bal	int
	revol_util	float
	total_acc	int
	initial_list_status	str
	out_prncp	float
	out_prncp_inv	float
	total_pymnt	float
	total_pymnt_inv	float
	total_rec_prncp	float
	total_rec_int	float
	total_rec_late_fee	float
	recoveries	float
	collection_recovery_fee	float
	last_pymnt_d	str
	last_pymnt_amnt	float
	next_pymnt_d	str
	last_credit_pull_d	str
	collections_12_mths_ex_med	int
	mths_since_last_major_derog	str
	policy_code	int
	not_compliant	in

Reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan. You should have code analogous to

In [6]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: +1 if (x==0) else -1)

We will only be considering these four features:

In [7]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [8]:
loans = loans[features + [target]]

In [9]:
loans.head(5)

grade,term,home_ownership,emp_length,safe_loans
B,36 months,RENT,10+ years,1
C,60 months,RENT,< 1 year,-1
C,36 months,RENT,10+ years,1
C,36 months,RENT,10+ years,1
A,36 months,RENT,3 years,1


Subsample dataset to make sure classes are balanced

In [10]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.502236174422
Percentage of risky loans                : 0.497763825578
Total number of loans in our new dataset : 46508


Apply one-hot encoding

In [11]:
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    
    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

Perform train/validation split

In [12]:
train_data, test_data = loans_data.random_split(0.8, seed=1)

Decision tree implementation

In [13]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

In [14]:
loans_data['grade.A'].sum()

6422L

Function to count number of mistakes while predicting majority class

In [15]:
def intermediate_node_num_mistakes(labels_in_node):
    if (len(labels_in_node) == 0):
        return 0
    
    num_safe_loans = len(labels_in_node[labels_in_node == +1])
    num_bad_loans = len(labels_in_node[labels_in_node == -1])
    
    if(num_safe_loans > num_bad_loans):
        num_mistakes = num_bad_loans
    else:
        num_mistakes = num_safe_loans
    
    return num_mistakes

In [16]:
# Test case 1
example_labels = np.array([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 1 failed... try again!'

# Test case 2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 3 failed... try again!'
    
# Test case 3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 3 failed... try again!'

Test passed!
Test passed!
Test passed!


Function to pick best feature to split on

In [17]:
def best_splitting_feature(data, features, target):
    
    best_feature = None
    least_error = 2
    
    total_obs = len(data)
    
    for feature in features:
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        
        left_split_error = intermediate_node_num_mistakes(left_split[target])
        right_split_error = intermediate_node_num_mistakes(right_split[target])
        
        error = (left_split_error + right_split_error) / total_obs
        
        if(error < least_error):
            best_feature = feature
            least_error = error
    
    return best_feature

In [18]:
if best_splitting_feature(train_data, features, 'safe_loans') == 'term. 36 months':
    print 'Test passed!'
else:
    print 'Test failed... try again!'

Test passed!


Building the tree

In [19]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}   ## YOUR CODE HERE 
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1         ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1         ## YOUR CODE HERE        

    # Return the leaf node
    return leaf 

In [20]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if (intermediate_node_num_mistakes(target_values) == 0):  ## YOUR CODE HERE
        print "Stopping condition 1 reached."     
        # If no mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if (remaining_features == None):   ## YOUR CODE HERE
        print "Stopping condition 2 reached."    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if (current_depth >= max_depth):  ## YOUR CODE HERE
        print "Reached maximum depth. Stopping for now."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    splitting_feature = best_splitting_feature(data, remaining_features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]     ## YOUR CODE HERE
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "Creating right node."
        ## YOUR CODE HERE
        return create_leaf(right_split[target])

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [21]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [22]:
small_data_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_data_decision_tree)
    print 'Number of nodes that should be there : 13' 

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (1048 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length.n/a. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points)

Build the tree!

Train a tree model on the train_data. Limit the depth to 6 (max_depth = 6) to make sure the algorithm doesn't run for too long. Call this tree my_decision_tree. Warning: The tree may take 1-2 minutes to learn.

In [23]:
my_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade.C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade.D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade.E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

In [24]:
def classify(tree, x, annotate = False):
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction']
    else:        
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            if annotate:
                print "...left split"
            return classify(tree['left'], x, annotate)
        else:
            if annotate:
                print "...right split"
            return classify(tree['right'], x, annotate)

In [25]:
print test_data[0]

{'emp_length.3 years': 0L, 'home_ownership.RENT': 1L, 'home_ownership.OWN': 0L, 'emp_length.6 years': 0L, 'emp_length.9 years': 0L, 'emp_length.1 year': 0L, 'home_ownership.OTHER': 0L, 'safe_loans': -1L, 'emp_length.< 1 year': 0L, 'emp_length.10+ years': 0L, 'emp_length.5 years': 0L, 'term. 60 months': 1L, 'home_ownership.MORTGAGE': 0L, 'emp_length.2 years': 1L, 'emp_length.7 years': 0L, 'emp_length.n/a': 0L, 'grade.D': 1L, 'grade.E': 0L, 'grade.F': 0L, 'grade.G': 0L, 'grade.A': 0L, 'grade.B': 0L, 'grade.C': 0L, 'emp_length.4 years': 0L, 'term. 36 months': 0L, 'emp_length.8 years': 0L}


In [26]:
print 'Predicted class: %s ' % classify(my_decision_tree, test_data[0])

Predicted class: -1 


# What was the feature that my_decision_tree first split on while making the prediction for test_data[0]?

# What was the first feature that lead to a right split of test_data[0]?

# What was the last feature split on before reaching a leaf node for test_data[0]?

In [27]:
classify(my_decision_tree, test_data[0], annotate=True)

Split on term. 36 months = 0
...left split
Split on grade.A = 0
...left split
Split on grade.B = 0
...left split
Split on grade.C = 0
...left split
Split on grade.D = 1
...right split
At leaf, predicting -1


-1

In [28]:
def evaluate_classification_error(tree, data, target='safe_loans'):
    prediction = data.apply(lambda x: classify(tree, x, False))    
    num_err = np.sum(np.array(prediction != data[target]))
    total_num_points = len(data)
    classification_error = (num_err / total_num_points)
    return classification_error

# Rounded to 2nd decimal point, what is the classification error of my_decision_tree on the test_data?

In [29]:
evaluate_classification_error(my_decision_tree, test_data)

0.38377854373115039

Printing out a decision stump

In [30]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print "(leaf, label: %s)" % tree['prediction']
        return None
    split_feature, split_value = split_name.split('.')
    print '                       %s' % name
    print '         |---------------|----------------|'
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '  [{0} == 0]               [{0} == 1]    '.format(split_name)
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree'))

# What is the feature that is used for the split at the root node?

In [31]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


We can print out the left subtree by running the code

In [32]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


We can similarly print out the left subtree of the left subtree of the root by running the code

In [33]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade.A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.B == 0]               [grade.B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


# What is the path of the first 3 feature splits considered along the left-most branch of my_decision_tree?

In [34]:
print_stump(my_decision_tree)
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])
print_stump(my_decision_tree['left']['left']['left'], my_decision_tree['left']['left']['splitting_feature'])

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
                       grade.A
         |---------------|----------------|
         |                    

# What is the path of the first 3 feature splits considered along the right-most branch of my_decision_tree?

In [35]:
print_stump(my_decision_tree)
print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.D == 0]               [grade.D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)
